# CRISP-DM Bullshit here

### Import librairies

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# from functions import *
from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn import tree
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from sklearn.preprocessing import OneHotEncoder
from IPython.display import Image  
from pydotplus import graph_from_dot_data
from pprint import pprint

%matplotlib inline

### Read csv file and preview file

In [7]:
df = pd.read_csv("Poker.csv", encoding='latin-1')
df.head(2)

,Player Name,Site,Hands,Net Won,VP$IP,PFR,3Bet,Postflop\nAgg%,W$WSF%,WTSD%,...,Raise\nRiver CBet,Squeeze,Raise Two\nRaisers,Call Two\nRaisers,vs 3Bet\nFold,vs 3Bet\nCall,vs 3Bet\nRaise,vs 4Bet\nFold,vs 4Bet\nCall,vs 4Bet\nRaise
0,LocSta,22,2035485,128473.89,0.249043,0.210084,0.094206,0.351073,0.477881,0.293348,...,0.113415,0.102155,0.040503,0.016301,0.604096,0.242094,0.154423,0.354854,0.284278,0.360868
1,.PARTOUCHE.,22,178311,15940.79,0.251039,0.201261,0.077332,0.314294,0.422270,0.242501,...,0.135371,0.076993,0.026232,0.010154,0.679106,0.209865,0.112322,0.481436,0.191832,0.326733


In [8]:
print("Before filtering")
winners = len(df[df["Net Won"] > 0])
losers = len(df[df["Net Won"] <= 0])
print(winners, "winners")
print(losers, "losers")
print("Winners %", winners / (winners + losers))

Before filtering
3085 winners
5737 losers
Winners % 0.3496939469508048


### If we want to use only the players with a minimum number of hands uncomment the cell below

In [9]:
# df = df[df['Hands'] > 500]

In [10]:
pprint(list(df.columns))

['Player Name',
 'Site',
 'Hands',
 'Net Won',
 'VP$IP',
 'PFR',
 '3Bet',
 'Postflop\nAgg%',
 'W$WSF%',
 'WTSD%',
 'Won $\nat SD',
 'Flop\nCBet%',
 'Turn\nCBet%',
 'River\nCBet%',
 'Fold to\nFlop Cbet',
 'Fold to\nTurn CBet',
 'Fold to\nRiver CBet',
 'Raise\nFlop Cbet',
 'Raise\nTurn CBet',
 'Raise\nRiver CBet',
 'Squeeze',
 'Raise Two\nRaisers',
 'Call Two\nRaisers',
 'vs 3Bet\nFold',
 'vs 3Bet\nCall',
 'vs 3Bet\nRaise',
 'vs 4Bet\nFold',
 'vs 4Bet\nCall',
 'vs 4Bet\nRaise']


### Rename columns for more clarity (replacing \n by spaces)

In [11]:
df.columns = df.columns.str.replace('\n',' ')
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns

In [12]:
df.head(2)

,Player_Name,Site,Hands,Net_Won,VP$IP,PFR,3Bet,Postflop_Agg%,W$WSF%,WTSD%,...,Raise_River_CBet,Squeeze,Raise_Two_Raisers,Call_Two_Raisers,vs_3Bet_Fold,vs_3Bet_Call,vs_3Bet_Raise,vs_4Bet_Fold,vs_4Bet_Call,vs_4Bet_Raise
0,LocSta,22,2035485,128473.89,0.249043,0.210084,0.094206,0.351073,0.477881,0.293348,...,0.113415,0.102155,0.040503,0.016301,0.604096,0.242094,0.154423,0.354854,0.284278,0.360868
1,.PARTOUCHE.,22,178311,15940.79,0.251039,0.201261,0.077332,0.314294,0.422270,0.242501,...,0.135371,0.076993,0.026232,0.010154,0.679106,0.209865,0.112322,0.481436,0.191832,0.326733


### Build the target based on Net Won vs Hands Played

In [13]:
df["target"] = df["Net_Won"] > 0

### Drop useless columns

In [14]:
df.drop(['Player_Name', 'Site', "Hands", "Net_Won"], axis=1, inplace=True)

## Pre-processing data

#### Check data types

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8822 entries, 0 to 8821
Data columns (total 26 columns):
VP$IP                 8822 non-null float64
PFR                   8822 non-null float64
3Bet                  8822 non-null float64
Postflop_Agg%         8822 non-null float64
W$WSF%                8822 non-null float64
WTSD%                 8822 non-null float64
Won_$_at_SD           8822 non-null float64
Flop_CBet%            8822 non-null float64
Turn_CBet%            8822 non-null float64
River_CBet%           8822 non-null float64
Fold_to_Flop_Cbet     8822 non-null float64
Fold_to_Turn_CBet     8822 non-null float64
Fold_to_River_CBet    8822 non-null float64
Raise_Flop_Cbet       8822 non-null float64
Raise_Turn_CBet       8822 non-null float64
Raise_River_CBet      8822 non-null float64
Squeeze               8822 non-null float64
Raise_Two_Raisers     8822 non-null float64
Call_Two_Raisers      8822 non-null float64
vs_3Bet_Fold          8822 non-null float64
vs_3Bet_Call 

All our predictors are floats (percentages), there are no missing values and our target is a boolean

#TODO: Check for number of unique values in the last 15 columns and maybe drop the ones that don't have a lot of different values meaning there were not enough occurences

### Check for class imbalance

In [16]:
winners = len(df[df["target"] == True])
losers = len(df[df["target"] == False])
print(winners, "winners")
print(losers, "losers")
print("Winners %", winners / (winners + losers))
print("Losers %", 1 - (winners / (winners + losers)))

3085 winners
5737 losers
Winners % 0.3496939469508048
Losers % 0.6503060530491952


The class imbalance isn't very strong, therefore we can move on with this data

### Because we don't have too many dimensions, instead of using Principal Component Analysis we will check for multicollinearities over 70 percents using our homemade function

### Check multicollinearities

In [24]:
print_corr(df, pct=0.7)

NameError: name 'print_corr' is not defined

None of our features have more than 70 percent multicollinearities

### Split the data between train and test datasets

In [18]:
labels = df["target"]
df = df.drop(["target"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.20, random_state=42)
X_train = X_train.reset_index().drop(["index"], axis=1)
y_train = y_train.reset_index().drop(["index"], axis=1)
X_test = X_test.reset_index().drop(["index"], axis=1)
y_test = y_test.reset_index().drop(["index"], axis=1)

## Baseline classifier using Logistic Regression

We are going to build a Logistic Regression model using a 5 fold cross validation as our baseline model

In [19]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

7057
7057
1765
1765


In [35]:
# Logistic Regression Cross Validation
baseline = LogisticRegressionCV(random_state=42, cv=5)

# Fit the pipelines
baseline.fit(X_train, y_train)

baseline = round(baseline.score(X_train, y_train), 4)

print("Our baseline model score is", baseline)
# print(np.sum(baseline.predict(X_test) == y_test)/len(y_test))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Our baseline model score is 0.7527


# Feature Engineering

## Interactions

In [45]:
labels  = labels*1

In [54]:
from itertools import combinations
from sklearn.model_selection import cross_val_score

regression = LogisticRegression()

combs = list(combinations(list(X_train.columns), 2))
interactions = []
data = df.copy()
for comb in combs:
    i = 0
    i +=1
    colname = 'int' +str(i)
    data[colname] = data[comb[0]] * data[comb[1]]
    score = np.mean(cross_val_score(regression, data, labels, scoring='r2'))
    
    if score > baseline: interactions.append((comb[0], comb[1], round(score,3)))
            
print('Top 3 interactions: %s' %sorted(interactions, key=lambda inter: inter[2], reverse=True)[:5])


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will ch

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will ch

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will ch

...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will ch

Top 3 interactions: []


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to

In [52]:
print(interactions)

[]


In [17]:
X_train.head()

,VP$IP,PFR,3Bet,Postflop_Agg%,W$WSF%,WTSD%,Won_$_at_SD,Flop_CBet%,Turn_CBet%,River_CBet%,...,Raise_River_CBet,Squeeze,Raise_Two_Raisers,Call_Two_Raisers,vs_3Bet_Fold,vs_3Bet_Call,vs_3Bet_Raise,vs_4Bet_Fold,vs_4Bet_Call,vs_4Bet_Raise
0,0.217742,0.165323,0.043478,0.339623,0.461538,0.461538,0.583333,0.666667,0.500000,0.000000,...,0.000000,0.058824,0.000000,0.000000,0.714286,0.142857,0.142857,0.000000,0.000000,0.000000
1,0.529730,0.183784,0.026316,0.588710,0.590909,0.181818,0.500000,0.714286,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.250000,0.250000,0.750000,0.000000,0.000000,0.000000,0.000000
2,0.367943,0.149903,0.083039,0.363300,0.553086,0.239506,0.453608,0.850000,0.714286,0.363636,...,0.125000,0.037736,0.023256,0.000000,0.666667,0.296296,0.037037,0.400000,0.400000,0.200000
3,0.260417,0.159722,0.056000,0.345455,0.436364,0.290909,0.437500,0.500000,0.800000,1.000000,...,0.000000,0.166667,0.000000,0.083333,0.000000,0.875000,0.125000,0.000000,1.000000,0.000000
4,0.272482,0.202425,0.080512,0.251121,0.431767,0.259508,0.560345,0.525974,0.321429,0.500000,...,0.142857,0.024793,0.024691,0.024691,0.404762,0.464286,0.130952,0.473684,0.263158,0.263158


## Polynomials

In [18]:
print(np.any(np.isnan(X_train)))
print(np.all(np.isfinite(X_train)))

False
True


In [35]:
# columns = list(X_train.columns)
# for col in columns:
#     for degree in [2, 3, 4]:
#         #Copy the X_train set
#         X_ = X_train.copy()
        
#         #Instantiate PolynomialFeatures
#         poly = PolynomialFeatures(degree, include_bias=False)

#         #Fit and transform X_train
#         X_poly_train = poly.fit_transform(np.array(X_[col]).reshape(-1, 1))
#         X_ = pd.concat([X_.drop(col, axis=1),pd.DataFrame(X_poly_train)], axis=1)

#         #Instantiate and fit a cross validation logistic regression model to the polynomial transformed train features
#         lr = LogisticRegressionCV(random_state=42, cv=5)
#         reg_poly = lr.fit(X_, y_train)

#         #Get predicted values for transformed polynomial train data
#         y_pred = reg_poly.predict(X_)
#         y_pred = np.array(y_pred).reshape(-1, 1)
#         print("y train shape",y_train.shape)
#         print("y pred shape",y_pred.shape)
#         #Calculate Score
#         score = lr.score(y_train, y_pred)
        
#         polynomials.append((col, degree, round(score, 3)))
# print('Top 10 polynomials: %s' %sorted(polynomials, key=lambda poly: poly[2], reverse=True)[:10])

In [ ]:
# We'll fit 3 different polynomial regression models from degree 2 to degree 4
columns = list(X_train.columns)
for col in columns:
    for degree in ([2, 3, 4]):

        # Instantiate PolynomialFeatures
        poly = PolynomialFeatures(degree)

        # Fit and transform X_train
        X_poly_train = poly.fit_transform(X_train)

        # Instantiate and fit a linear regression model to the polynomial transformed train features
        reg_poly = LinearRegression().fit(X_poly_train, y_train)

        # Transform the test data into polynomial features
        X_poly_test = poly.transform(X_test)

        # Get predicted values for transformed polynomial test data  
        y_pred = reg_poly.predict(X_poly_test)

        # Evaluate model performance on test data
        print("degree %d" % degree, r2_score(y_test, y_pred))

        # Transform the full data
        X_poly = poly.transform(X)

        # Now, we want to see what the model predicts for the entire data  
        y_poly = reg_poly.predict(X_poly)

### Split the data between train, validation and test datasets

In [ ]:
labels = df["target"]
df = df.drop(["target"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.20, random_state=42)

### Scale using min max scaling ?

In [ ]:
# # Instantiate StandardScaler
# scaler = StandardScaler()
# # MinMaxScaler(feature_range=(0, 1), copy=True)
# # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

# # Transform the training and test sets
# scaled_data_train = scaler.fit_transform(X_train)
# scaled_data_test = scaler.transform(X_test)

# # Convert into a DataFrame
# scaled_df_train = pd.DataFrame(scaled_data_train, columns=one_hot_df.columns)
# scaled_df_train.head()

## Pipelines

In [ ]:
# # Construct some pipelines
# pipe_svm = Pipeline([('pca', PCA(n_components=27)),
#         ('clf', svm.SVC(random_state=123))])
        
# pipe_tree = Pipeline([('pca', PCA(n_components=27)),
#         ('clf', tree.DecisionTreeClassifier(random_state=123))])

# pipe_rf = Pipeline([('pca', PCA(n_components=27)),
#         ('clf', RandomForestClassifier(random_state=123))])

# # List of pipelines, List of pipeline names
# pipelines = [pipe_svm, pipe_tree, pipe_rf]
# pipeline_names = ['Support Vector Machine','Decision Tree','Random Forest']

# # Loop to fit each of the three pipelines
# for pipe in pipelines:
#     print(pipe)
#     pipe.fit(X_train, y_train)

# # Compare accuracies
# for index, val in enumerate(pipelines):
#     print('%s pipeline test accuracy: %.3f' % (pipeline_names[index], val.score(X_test, y_test)))

## Pipelines with gridsearch

In [ ]:
# # ⏰ This cell may take a long time to run!
# # Construct pipeline
# pipe_rf = Pipeline([('pca', PCA(n_components=27)),
#             ('clf', RandomForestClassifier(random_state = 123))])

# # Set grid search params
# param_grid_forest = [ 
#   {'clf__n_estimators': [120],
#    'clf__criterion': ['entropy', 'gini'], 
#    'clf__max_depth': [4, 5, 6],  
#    'clf__min_samples_leaf':[0.05 ,0.1, 0.2],  
#    'clf__min_samples_split':[0.05 ,0.1, 0.2]
#   }
# ]

# # Construct grid search
# gs_rf = GridSearchCV(estimator=pipe_rf,
#             param_grid=param_grid_forest,
#             scoring='accuracy',
#             cv=3, verbose=2, return_train_score = True)

# # Fit using grid search
# gs_rf.fit(X_train, y_train)

# # Best accuracy
# print('Best accuracy: %.3f' % gs_rf.best_score_)

# # Best params
# print('\nBest params:\n', gs_rf.best_params_)

## Running logistic regression with different thresholds

## Run KNNs

In [ ]:
# Instantiate KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit the classifier
clf.fit(scaled_data_train, y_train)

# Predict on the test set
test_preds = clf.predict(scaled_data_test)

# evaluation metrics for the test predictions stored in test_preds, and the corresponding labels in y_test.
print_metrics(y_test, test_preds)

# Finding the best K
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25)

For further tuning, you can use scikit-learn's built-in GridSearch() to perform a similar exhaustive check of hyperparameter combinations and fine tune model performance. For a full list of model parameters, see the sklearn documentation !

## Run SVMs

https://github.com/learn-co-students/dsc-building-an-svm-using-scikit-learn-lab-london-ds-111819/tree/solution

Kernel Trick
https://github.com/learn-co-students/dsc-kernels-in-scikit-learn-lab-london-ds-111819/tree/solution

## Run Decision Trees

In [ ]:
# Create the classifier, fit it on the training data and make predictions on the test set
# clf = DecisionTreeClassifier(criterion='entropy')

# clf.fit(X_train, y_train)

## Run Clustering

## Run Ensemble Methods

# Conclusions